In [56]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

## **Hand Project**

Create a program that can detect a hand, segment it and count the number of fingers being held up.

In [57]:
# Updates a moving average of the background values in a region of interest.
# When hand enters the ROI, we can detect a change and apply thresholding.

background = None
roi_x1, roi_y1, roi_x2, roi_y2 = 0, 0, 400, 400


def moving_avg(frame, accumulated_weight=0.5):
    global background

    if background is None:
        background = frame.copy().astype(np.float32)
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)  # type: ignore


def segment(frame, threshold=25):
    global background

    diff = cv2.absdiff(background.astype(np.uint8), frame)  # type: ignore
    ret, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not len(contours):
        return None

    # Assuming the largest contour is the hand.
    hand_segment = max(contours, key=cv2.contourArea)

    return thresholded, hand_segment


def count_fingers(thresholded, hand_segment):
    hull_points = np.squeeze(cv2.convexHull(hand_segment))  # n x 2

    top = hull_points[np.argmin(hull_points[:, 1])]
    bottom = hull_points[np.argmax(hull_points[:, 1])]
    left = hull_points[np.argmin(hull_points[:, 0])]
    right = hull_points[np.argmax(hull_points[:, 0])]

    cx = (left[0] + right[0]) // 2
    cy = (top[1] + bottom[1]) // 2

    center = np.array([cx, cy], ndmin=2)
    extreme_points = np.array([left, right, top, bottom], ndmin=2)

    dist = euclidean_distances(center, extreme_points)
    radius = int(0.9 * dist.max())
    circumference = 2 * np.pi * radius

    circular_roi = np.zeros_like(thresholded[:2], dtype=np.uint8)
    cv2.circle(circular_roi, (cx, cy), radius, 255, 10)  # type: ignore
    contours, hierarchy = cv2.findContours(
        circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    count = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        out_of_wrist = (cy + cy * 0.25) > (y + h)
        large_enough = circumference * 0.25 > contour.shape[0]
        if out_of_wrist and large_enough:
            count += 1

    return count

In [59]:
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    success, frame = cap.read()
    if not success:
        break

    frame_copy = frame.copy()
    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    if num_frames < 60:
        moving_avg(gray)
    else:
        hand = segment(gray)
        if hand is not None:
            thresholded, hand_segment = hand
            cv2.drawContours(frame_copy, [hand_segment + (roi_x1, roi_y1)], -1, (255, 0, 0), 5)
            fingers = count_fingers(thresholded, hand_segment)
            cv2.putText(
                frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2
            )
            cv2.imshow("Thresholded", thresholded)

    cv2.rectangle(frame_copy, (roi_x1, roi_y1), (roi_x2, roi_y2), (0, 255, 0), 5)
    num_frames += 1

    cv2.imshow("Finger Count", frame_copy)
    if cv2.waitKey(1) == ord("q"):
        break


cap.release()
cv2.destroyAllWindows()